In [19]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [20]:
import uuid
import time

def generate_face_id(num_char=8):
    return uuid.uuid4().hex[:num_char]

In [21]:
import torch
from torchvision import transforms
import numpy as np
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1
from IPython.display import display

def image_to_face(image, min_face_size=20, threshold_p_val=0.9, device='cpu'):
    mtcnn = MTCNN(keep_all=True, min_face_size=min_face_size, device=device)
    transform = transforms.ToTensor()

    bounds, p_vals = mtcnn.detect(image)

    threshold_p_val = threshold_p_val
    valid_bounds = []
    valid_p_vals = []
    if bounds is not None:
        for box, p_val in zip(bounds, p_vals):
            if p_val > threshold_p_val:
                valid_bounds.append(box)
                valid_p_vals.append(p_val)

    face_tensors = []
    boundary_boxs = []
    if valid_bounds is not None:
        for i, box in enumerate(valid_bounds):
            (box[0], box[1], box[2], box[3]) = (box[0] - 20, box[1] - 30,
                                                     box[2] + 20, box[3] + 20)
            face = image.crop(tuple(box))
            face_tensor = transform(face).to(device)

            face_tensors.append(face_tensor)
            boundary_boxs.append(box)

    return face_tensors, boundary_boxs, valid_p_vals

In [22]:
def face_tensor_to_embedding(face_tensor, resnet_model='vggface2', device='cpu'):
    resnet = InceptionResnetV1(pretrained=resnet_model, device=device).eval()

    with torch.inference_mode():
        face_embedding = resnet(face_tensor.unsqueeze(0))

    return face_embedding.cpu()

In [23]:
def image_to_face_list(image_path):
    image = Image.open(image_path)

    face_list = []

    face_tensors, bounds, p_vals = image_to_face(image)

    for i, face_tensor in enumerate(face_tensors):
        face_embedding = face_tensor_to_embedding(face_tensor)
        face_id = generate_face_id()

        face_dict = dict()

        face_dict['id'] = face_id
        face_dict['embedding'] = face_embedding
        face_dict['boundary_box'] = bounds[i]
        face_dict['probability'] = p_vals[i]

        face_list.append(face_dict)

    return face_list

{'id': '6619e3c5', 'embedding': tensor([[ 7.5517e-02,  1.4039e-02,  5.2241e-04, -1.6453e-02, -3.1299e-02,
          6.3861e-03,  1.4869e-02,  2.9185e-02,  1.7684e-02, -4.6652e-02,
         -2.2446e-02, -2.3610e-02,  1.2832e-02, -2.8295e-02,  6.2884e-04,
          7.9983e-03,  3.8094e-02, -1.1548e-02,  2.4508e-03,  4.0417e-02,
         -3.3536e-02, -7.0266e-03, -5.0034e-03, -3.9938e-02,  2.9967e-02,
          2.4181e-02,  9.2235e-03, -1.7886e-02,  1.9677e-02, -1.6586e-02,
          3.0230e-02,  2.7037e-02, -5.8918e-02,  2.4788e-02,  2.0281e-02,
         -2.5391e-02, -3.5062e-02, -4.9145e-02,  8.0737e-03,  3.0518e-04,
          8.3124e-02, -2.6416e-02,  1.6551e-02,  9.5460e-03, -8.0191e-02,
         -4.3238e-02, -8.3784e-02, -2.3507e-02, -1.8218e-02, -3.6323e-02,
          4.7534e-03, -1.8465e-02, -3.1667e-02,  1.1789e-02, -5.7583e-02,
         -8.3763e-04,  5.2924e-03, -9.9782e-02,  1.8727e-03,  1.4004e-02,
         -6.6594e-03,  1.1273e-01, -2.0360e-02,  7.7518e-02,  2.0711e-02,
      

In [ ]:
image_to_face_tensors()